In [4]:
print("Hello, World!")

Hello, World!


In [10]:
class Holding:
    def __init__(self, name, date, shares, price):
        self.date = date
        self.name = name
        self.price = price
        self.shares = shares

In [11]:
h: Holding = Holding('AA', '2007-06-11', 100, 32.2)

In [12]:
h

# \_\_repr__
Above is not a very nice REPResentation of the instance *h* of class *holdings*. (See what I did there?)
But can we make it better? Of course we can! Let's add method \_\_repr__ to our class. This method used exactly for the purpose. To help (mostly developers) to see a more meaningful representation of an object.


In [24]:
class Holding:
    def __init__(self, name, date, shares, price):
        self.date = date
        self.name = name
        self.price = price
        self.shares = shares
        
    def __repr__(self):
        return f'Holding {self.name} dated {self.date} with {self.shares} shares at price {self.price}'

In [25]:
h = Holding('AA', '2007-06-11', 100, 32.2)

In [26]:
h

Holding AA dated 2007-06-11 with 100 shares at price 32.2

In [31]:
h.__dict__

{'date': '2007-06-11', 'name': 'AA', 'price': 32.2, 'shares': 100}

# dot operator
*Every* instance holds all it's variables inside a dictionary. That means what you're doing wiht \_\_init__ function it populates a dictionary. Knowing that you can understand how the dot (.) operator works. 

In [33]:
h.shares

100

In [34]:
h.__dict__['shares']

100

In [35]:
h.__dict__['shares'] = 150

In [36]:
h.shares

150

In [37]:
h.__dict__

{'date': '2007-06-11', 'name': 'AA', 'price': 32.2, 'shares': 150}

But if variables are stored inside a dictionary of an instance, where do the methods of an instance being stored?

Those functions are actually part of the class from which *h* was instantiated.

In [55]:
class Holding:
    def __init__(self, name, date, shares, price):
        self.date = date
        self.name = name
        self.price = price
        self.shares = shares
        
    def __repr__(self):
        return f'Holding {self.name} dated {self.date} with {self.shares} shares at price {self.price}'
    
    def cost(self):
        return self.shares * self.price

In [56]:
h.__class__.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'Holding' objects>,
              '__doc__': None,
              '__init__': <function __main__.Holding.__init__>,
              '__module__': '__main__',
              '__repr__': <function __main__.Holding.__repr__>,
              '__weakref__': <attribute '__weakref__' of 'Holding' objects>})

In [57]:
Holding.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'Holding' objects>,
              '__doc__': None,
              '__init__': <function __main__.Holding.__init__>,
              '__module__': '__main__',
              '__repr__': <function __main__.Holding.__repr__>,
              '__weakref__': <attribute '__weakref__' of 'Holding' objects>,
              'cost': <function __main__.Holding.cost>})

In [58]:
h.__class__.__dict__ == Holding.__dict__

False

In [60]:
set(Holding.__dict__) - set(h.__class__.__dict__)

{'cost'}

When you write cost() it actually calls. It looks up the function inside class definition and passes instances of a class to it.

In [64]:
Holding.__dict__['cost'](h)

4830.0

# Preventing unintended usage of variables inside the class

### Attempt 1 (C++ style getters and setters)

In [67]:
class Holding:
    def __init__(self, name, date, shares, price):
        self.date = date
        self.name = name
        self.price = price
        self.shares = shares
        
    def get_price(self):
        return self.price
    
    def set_price(self, new_price):
        if not isinstance(new_price, float):
            self.price = new_price

In [68]:
h = Holding('AA', '2007-06-11', 100, 32.2)

In [73]:
# but now instead of regular h.price 
# you have to enforce everyone to use 

h.get_price()
h.set_price(5)

# this is not gonna happen


### Attempt 2 (making variable "private") 

Side note: there is no real "private" in Python as there is private in C++ or Java. 


in Python _variable_name only hints to the user that this variable is for private usage.


But nothing can stop this user from misbehaving

In [78]:
class Holding:
    def __init__(self, name, date, shares, price):
        self.date = date
        self.name = name
        self._price = price
        self.shares = shares
        
    def get_price(self):
        return self._price
    
    def set_price(self, new_price):
        if not isinstance(new_price, float):
            self._price = new_price

In [79]:
h = Holding('AA', '2007-06-11', 100, 32.2)

In [82]:
h._price = 5

In [83]:
h._price

5

But the actual variable name is *_price*, not just *price*. Still doesn't solve our problem. There must be a better way!

# Attempt 3 (property!)

In [93]:
class Holding:
    def __init__(self, name, date, shares, price):
        self.date = date
        self.name = name
        self._price = price
        self.shares = shares
        
    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self, new_price):
        if not isinstance(new_price, float):
            raise TypeError('Expected float')
        self._price = new_price

In [94]:
h = Holding('AA', '2007-06-11', 100, 32.2)

In [95]:
h.price

32.2

In [97]:
# This line will generate an error. Because we are trying to set price to a string instead of a float.
h.price = 'a lot'

TypeError: Expected float

You don't have to change any existing code with this approach! 

# But how exactly dot works on a property?

In [98]:
# first we consult the class - if it has such a property
h.__class__.__dict__['price']

In [99]:
# so we have a property! But does it have a get method?
hasattr(h.__class__.__dict__['price'], '__get__')

True

In [103]:
# turns out that it does! Now Python can fire it for the instance *h* that we are working with
h.__class__.__dict__['price'].__get__(h)

32.2

In [104]:
# set works exactly the same way, it just asks it the property has a __set__ method
hasattr(h.__class__.__dict__['price'], '__set__')

True

Now we can use our newfound knowledge.

In [110]:
class Integer:
    """
    This type of class is known as a descriptor.
    It type-checks incoming variables.
    """
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.name]
        
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected int')
        instance.__dict__[self.name] = value
       
       
class Point:
    x = Integer('x')  # this initialisation has to be here
    y = Integer('y')  # it has to be used at the class level
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [111]:
p = Point(5, 3)  # this works just fine

In [112]:
p = Point(5.2, 3)  # this doesn't work because we tried to set float to a variable which suppose to be an int

TypeError: Expected int

# Inheritance
But what if you want a typed-variable for a float and str? You can make this class even more general

In [120]:
class Typed:
    expected_type = object
    
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.name]
        
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError(f'Expected {self.expected_type}')
        instance.__dict__[self.name] = value


class Integer(Typed):
    expected_type = int


class Float(Typed):
    expected_type = float


class String(Typed):
    expected_type = str
    
    
class Point:
    x = Integer('x')
    y = Float('y')
    name = String('name')
    
    def __init__(self, x, y, name):
        self.x = x
        self.y = y
        self.name = name

In [123]:
p = Point(5, 3, 'my_point')  # this generates an error, because one of the types is inappropriate

TypeError: Expected <class 'float'>

In [124]:
p = Point(5, 3.2, 'my_point')  # this does not generate an error because all types are appropriate

In [140]:
def __getattr__(name): pass  # only called if an attribute is missing


def __setattr__(name, value): pass  # capture set attribute


In [141]:
# you can create a read-only class with this
class ReadOnly:
    def __init__(self, obj):
        self._obj = obj
    
    def __getattr__(self, name):
        return getattr(self._obj, name)
    
    def __setattr__(self, name, value):
        if name == '_obj':
            super().__setattr__(name, value)
        else:
            raise AttributeError('ReadOnly')


In [142]:
readonly_point = ReadOnly(p)

In [143]:
readonly_point.x

5

In [144]:
readonly_point.x = 6

AttributeError: ReadOnly

In [ ]:
# this can be used as inheritance
# because it forwards the requests


def __getattr__(self, name):
    return getattr(self.class_instance, name)